In [1]:
import requests
import pydicom
import json
from pathlib import Path
from urllib3.filepost import encode_multipart_formdata, choose_boundary
from azure.identity import DefaultAzureCredential, AzureCliCredential
from typing import List

# Authentication
Set up common variables

In [9]:
dicom_service_name = "dicomservice-eastus"
path_to_dicoms_dir = "./dicoms/"

# West US
# base_url = f"https://medicalimaging-ivan-images.dicom.azurehealthcareapis.com/v1" 

# East US
base_url = f"https://healthserviceseastusws-dicomservice-eastus.dicom.azurehealthcareapis.com/v1"

study_uid = "1.2.826.0.1.3680043.8.498.13230779778012324449356534479549187420"; #StudyInstanceUID for all 3 examples
series_uid = "1.2.826.0.1.3680043.8.498.45787841905473114233124723359129632652"; #SeriesInstanceUID for green-square and red-triangle
instance_uid = "1.2.826.0.1.3680043.8.498.47359123102728459884412887463296905395"; #SOPInstanceUID for red-triangle

In [38]:
from azure.identity import DefaultAzureCredential
credential = DefaultAzureCredential()

print(credential.credentials) # this can be used to find the index of the AzureCliCredential

(<azure.identity._credentials.environment.EnvironmentCredential object at 0x000002AD21205BA0>, <azure.identity._credentials.managed_identity.ManagedIdentityCredential object at 0x000002AD21205C00>, <azure.identity._credentials.shared_cache.SharedTokenCacheCredential object at 0x000002AD21207760>, <azure.identity._credentials.azure_cli.AzureCliCredential object at 0x000002AD212A0820>, <azure.identity._credentials.azure_powershell.AzurePowerShellCredential object at 0x000002AD212A0040>)


In [39]:
azureCliCred = None
for cred in credential.credentials:
    if isinstance(cred, AzureCliCredential):
        azureCliCred = cred
        break

if azureCliCred is None:
    print("No AzureCliCredential object found in the list.")
else:
    print("Selected AzureCliCredential object:", credential)

Selected AzureCliCredential object: <azure.identity._credentials.default.DefaultAzureCredential object at 0x000002AD21204130>


In [40]:
# need AzureCliCredential object here
token = azureCliCred.get_token('https://dicom.healthcareapis.azure.com')
bearer_token = f'Bearer {token.token}'
print(token.token)

eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6Ii1LSTNROW5OUjdiUm9meG1lWm9YcWJIWkdldyIsImtpZCI6Ii1LSTNROW5OUjdiUm9meG1lWm9YcWJIWkdldyJ9.eyJhdWQiOiJodHRwczovL2RpY29tLmhlYWx0aGNhcmVhcGlzLmF6dXJlLmNvbSIsImlzcyI6Imh0dHBzOi8vc3RzLndpbmRvd3MubmV0L2E0NjJmNDQ3LTc3Y2YtNDFiYy1hOTQ5LWQzNmI4NTg4NjM1MC8iLCJpYXQiOjE2Nzc2OTI5NzMsIm5iZiI6MTY3NzY5Mjk3MywiZXhwIjoxNjc3Njk3ODAxLCJhY3IiOiIxIiwiYWlvIjoiQVZRQXEvOFRBQUFBZmliQ2EzRklJN05WMDJwUTNVSHlLcVd2V0UxdHZDc1Zna3MzZ0VFVWlvZkQ1V2FqZ3h4bFBRaW8wTHFQaFNxVG84TXZpZGcxTlJKVlIwRzVzM1BtUmhzTFdrQVJVSkY5cmpFM2VLcG5lRkE9IiwiYWx0c2VjaWQiOiIxOmxpdmUuY29tOjAwMDM0MDAxOEFGMDIxOTYiLCJhbXIiOlsicHdkIl0sImFwcGlkIjoiMDRiMDc3OTUtOGRkYi00NjFhLWJiZWUtMDJmOWUxYmY3YjQ2IiwiYXBwaWRhY3IiOiIwIiwiZW1haWwiOiJ0YXJhcG92QGdtYWlsLmNvbSIsImZhbWlseV9uYW1lIjoiVGFyYXBvdiIsImdpdmVuX25hbWUiOiJJdmFuIiwiaWRwIjoibGl2ZS5jb20iLCJpcGFkZHIiOiI3MS4yMzEuMTc3LjUyIiwibmFtZSI6Ikl2YW4gVGFyYXBvdiIsIm9pZCI6Ijg4OGQ4YWU5LWI4MzYtNDgzMy04NmNmLTYyMWNlOTRmYjEwZiIsInB1aWQiOiIxMDAzMjAwMEM4RDNEQzJFIiwicmgiOiIwLkFWa0FSX1JpcE0

In [30]:
# Supporting method for multipart requests
def encode_multipart_related(fields, boundary=None):
    if boundary is None:
        boundary = choose_boundary()

    body, _ = encode_multipart_formdata(fields, boundary)
    content_type = str('multipart/related; boundary=%s' % boundary)

    return body, content_type

# Check if file is a DICOM file
def is_dicom_file(filename):
    try:
        dataset = pydicom.filereader.dcmread(filename, stop_before_pixels=True)
        return True
    except pydicom.errors.InvalidDicomError:
        return False

def human_readable_size(size_bytes):
    """
    Converts a number of bytes to a human-readable string that lists the number of KB, MB, GB, etc.
    depending on the magnitude of the number of bytes.
    """
    suffixes = ['B', 'Kb', 'Mb', 'Gb', 'Tb', 'Pb', 'Eb']  # list of suffixes for each size unit
    i = 0
    while size_bytes >= 1024 and i < len(suffixes)-1:
        size_bytes /= 1024
        i += 1
    return f"{size_bytes:.2f} {suffixes[i]}"

In [16]:
client = requests.session()


In [18]:
# Check that connection works and we can authorize to the server

headers = {"Authorization":bearer_token}
url= f'{base_url}/changefeed'
response = client.get(url,headers=headers)

# Change feed output can be quite verbose, limiting it to a few chars
print(response.content[0:200])

b'[{"sequence":1,"partitionName":"Microsoft.Default","studyInstanceUid":"1.2.826.0.1.3680043.8.498.13230779778012324449356534479549187420","seriesInstanceUid":"1.2.826.0.1.3680043.8.498.4578784190547311'


# Sending

## Batch send function

In [37]:
def send_dicom_files(path_to_dicom_dir: str, base_url: str, bearer_token: str, batch_size: int = 0, start_batch: int = 0):

    file_extension = ""

    # Get a list of all files in the folder with the specified file extension
    folder_path = Path(path_to_dicom_dir)
    file_names = [str(file_path) for file_path in folder_path.rglob(f'*{file_extension}') if Path.is_file(file_path)]

    print(f"Found {len(file_names)} files")

     # Create a list to store the file bytes
    rawfiles = []
    
    # Read the contents of each file and add them to the list
    for file_name in file_names:
        filepath = Path(path_to_dicom_dir).joinpath(file_name)
        # print(f"Loading {filepath}")
        if (is_dicom_file(filepath)):
            with open(filepath, 'rb') as reader:
                rawfile = reader.read()
            rawfiles.append((rawfile, len(rawfile)))
        else:
            print(f"Not a valid DICOM file, skipping: {file_name}")
    
    print(f"Loaded {len(rawfiles)} files, total: {human_readable_size(sum(f[1] for f in rawfiles))}")

    # Create a list of tuples containing the file data and MIME type
    files = []
    for i in range(len(rawfiles)):
        file_data = rawfiles[i][0]
        file_name = f'dicomfile{i}'
        file_mime_type = 'application/dicom'
        files.append({
                "size":rawfiles[i][1], 
                "payload": (file_name,('dicomfile', file_data, file_mime_type))
                })
    
    file_batches = [files]

    if (batch_size > 0):
        # Split the list of files into batches
        file_batches = [files[i:i+batch_size] for i in range(0, len(files), batch_size)]

    responses = []

    if (batch_size > 0):
        print(f"Preparing to send {len(file_batches)} batch{'' if len(file_batches)%10 == 1 else 'es'} of {batch_size} files each")
    else:
        print(f"Preparing to send {len(files)} files in one batch")

    if (start_batch > 0):
        print(f"Skipping to batch {start_batch}")

    files_sent = 0
    for index, file_batch in enumerate(file_batches):
        if (start_batch > 0 and index+1 < start_batch):
            continue

        # Encode the file data as multipart_related
        body, content_type = encode_multipart_related(fields=dict([f["payload"] for f in file_batch]))

        # Set the request headers
        headers = {
            'Accept': 'application/dicom+json',
            'Content-Type': content_type,
            'Authorization': bearer_token
        }        # Send the POST request to the API endpoint
        
        url = f'{base_url}/studies'
        response = client.post(url, data=body, headers=headers, verify=False)
        #response = lambda: None
        #response.status_code = 204

        responses.append(response)
        files_sent+=len(file_batch)
        print(f"Sent batch {index+1} of {len(file_batches)} with {len(file_batch)} files. Total {human_readable_size(sum(f['size'] for f in file_batch))} sent. Status: {response.status_code}")

    print(f"Sent {files_sent} files")

    # Return the response from the API endpoint
    return responses


## Send statement

In [42]:
responses = send_dicom_files(r"C:\temp\imaging\Jan23 - mri", base_url, bearer_token, 10)

Found 3034 files
Not a valid DICOM file, skipping: C:\temp\imaging\Jan23 - mri\kneeMRI2023.zip
Not a valid DICOM file, skipping: C:\temp\imaging\Jan23 - mri\kneeMRI2023\kneeMRI2023\autorun.dat
Not a valid DICOM file, skipping: C:\temp\imaging\Jan23 - mri\kneeMRI2023\kneeMRI2023\AutoRun.exe
Not a valid DICOM file, skipping: C:\temp\imaging\Jan23 - mri\kneeMRI2023\kneeMRI2023\autorun.inf
Not a valid DICOM file, skipping: C:\temp\imaging\Jan23 - mri\kneeMRI2023\kneeMRI2023\AutoRunH.dll
Not a valid DICOM file, skipping: C:\temp\imaging\Jan23 - mri\kneeMRI2023\kneeMRI2023\AutoRunV.dll
Not a valid DICOM file, skipping: C:\temp\imaging\Jan23 - mri\kneeMRI2023\kneeMRI2023\Contents.txt
Not a valid DICOM file, skipping: C:\temp\imaging\Jan23 - mri\kneeMRI2023\kneeMRI2023\desktop.ini
Not a valid DICOM file, skipping: C:\temp\imaging\Jan23 - mri\kneeMRI2023\kneeMRI2023\dRayViewer.exe
Not a valid DICOM file, skipping: C:\temp\imaging\Jan23 - mri\kneeMRI2023\kneeMRI2023\ExchangeServiceLogo.png
Not a

## Analyzing response

In [43]:
for index, response in enumerate(responses):
    print(f"Response {index+1} of {len(responses)}")

    jsondata = json.loads(response.content)
    print(f"Response classes: {len(jsondata)} ({[x for x in jsondata]})\n")

    failure_tag = "00081198"
    success_tag = "00081199"

    failure_reasons = {
        "272": "The store transaction didn't store the instance because of a general failure in processing the operation.",
        "43264": "The DICOM instance failed the validation.",
        "43265": "The provided instance StudyInstanceUID didn't match the specified StudyInstanceUID in the store request.",
        "45070": "A DICOM instance with the same StudyInstanceUID, SeriesInstanceUID, and SopInstanceUID has already been stored. If you wish to update the contents, delete this instance first.",
        "45071": "A DICOM instance is being created by another process, or the previous attempt to create has failed and the cleanup process hasn't had chance to clean up yet. Delete the instance first before attempting to create again."
    }

    if (failure_tag in jsondata):
        print(f"Failures ({failure_tag}):")
        # Parse the JSON text into a Python dictionary
        data = json.loads(response.content)[failure_tag]["Value"]

        # Count the number of objects
        count = len(data)

        # Print DICOM Error Comment for each object
        error_comments = []
        for index, item in enumerate(data):
            print(f"\nObj {index}, Instance UID {item['00081155']['Value'] if '00081155' in item else '[no iud]'}")
            failure_reason = str(item["00081197"]["Value"][0])
            print(f"Failure reason: {failure_reasons[failure_reason] if failure_reason in failure_reasons else f'Unknown ({failure_reason})'}")
            if '00741048' in item:
                for sub_item in item['00741048']['Value']:
                    if '00000902' in sub_item:
                        print(sub_item['00000902']['Value'])

        print(f"\nFailed objects: {count}\n-------")

    if (success_tag in jsondata):
        print(f"\nSuccesses ({success_tag}): {len(jsondata[success_tag]['Value'])}")


Response 1 of 24
Response classes: 2 (['00081198', '00081199'])

Failures (00081198):

Obj 0, Instance UID [no iud]
Failure reason: The DICOM instance failed the validation.
["DICOM100: (0010,0020) - The required tag '(0010,0020)' is missing."]

Failed objects: 1
-------

Successes (00081199): 9
Response 2 of 24
Response classes: 1 (['00081199'])


Successes (00081199): 10
Response 3 of 24
Response classes: 1 (['00081199'])


Successes (00081199): 10
Response 4 of 24
Response classes: 1 (['00081199'])


Successes (00081199): 10
Response 5 of 24
Response classes: 1 (['00081199'])


Successes (00081199): 10
Response 6 of 24
Response classes: 1 (['00081199'])


Successes (00081199): 10
Response 7 of 24
Response classes: 1 (['00081199'])


Successes (00081199): 10
Response 8 of 24
Response classes: 1 (['00081199'])


Successes (00081199): 10
Response 9 of 24
Response classes: 1 (['00081199'])


Successes (00081199): 10
Response 10 of 24
Response classes: 1 (['00081199'])


Successes (00081

# Deletion

Delete study

In [87]:
study_uid = "1.2.840.113619.2.81.290.1.33656.20140218.202536"

url = f'{base_url}/studies/{study_uid}'
response = client.delete(url, headers=headers) 

In [88]:
print(f"Status: {response.status_code}\n")
print(response.content)

Status: 204

b''
